In [5]:
import numpy as np
import tensorflow as tf
import nibabel as nib
import os

In [6]:
from infer_constants import CKPT_PATH,TEST_IMG

In [7]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
img_dir = "/home/jtabalon/airway_seg_data/test/ct/ct_patients10006S.nii"
patch_size = 64
weights_path = "/home/jtabalon/airway_seg/checkpoints/ckpt_airway_seg"


# Vars
patch_distance = int(patch_size / 2)

# Load image
img = nib.load(img_dir).get_fdata() / 2000.
img = img[:,:,0:512] # Ask Kyle about how to handle this... original size is (512,512,654)
                        # How do we handle the excess z axis? z coordinate varies from img to img
row_dim, column_dim, slice_dim  = img.shape[0], img.shape[1], img.shape[2]
patch_mid_row, patch_mid_col, patch_mid_slice = patch_distance,patch_distance,patch_distance

In [10]:
num_row_patchs, num_col_patchs, num_slice_patchs = int(row_dim / patch_size), \
                                                    int(column_dim / patch_size), \
                                                    int(slice_dim / patch_size)

In [11]:
model = tf.keras.models.load_model(weights_path, compile=False)


2022-04-21 08:38:10.714076: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-21 08:38:10.714623: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-21 08:38:10.776002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 08:38:10.777296: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2022-04-21 08:38:10.777308: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-21 08:38:10.778362: I tensorflow/stream_executor/platform/default/ds

In [12]:
predicted_mask = np.zeros(shape=(row_dim,column_dim,slice_dim))
counts = np.zeros(shape=(row_dim,column_dim,slice_dim))

In [15]:
count_patchs = 0
for slice in range(num_slice_patchs):
    for col in range(num_col_patchs):
        for patch in range(num_row_patchs):
            # Extract patch
            # print(f"slice: {slice}")
            # print(f"col: {col}")
            # print(f"patch: {patch}")
            print(f"patch midpoint location: {patch_mid_row, patch_mid_col, patch_mid_slice}")
            row_patch = img[(patch_mid_row-patch_distance):(patch_mid_row+patch_distance), \
                    (patch_mid_col-patch_distance):(patch_mid_col+patch_distance), \
                    (patch_mid_slice-patch_distance):(patch_mid_slice+patch_distance)]
            # Expand dims (necessary for model prediction)
            # print(f"patch shape: {np.shape(row_patch)}")
            expanded_row_patch = np.expand_dims(np.expand_dims(row_patch, -1), 0)
            # print(f"expanded patch shape: {np.shape(expanded_row_patch)}")
            # Make prediction
            with tf.device("/device:GPU:0"):
                inferred_patch = np.squeeze(model.predict(expanded_row_patch))
                # print(f"inferred patch shape: {np.shape(inferred_patch)}")
                print(f"patch midpoint location: {patch_mid_row, patch_mid_col, patch_mid_slice}")
                print("\n")
                predicted_mask[(patch_mid_row-patch_distance):(patch_mid_row+patch_distance), \
                        (patch_mid_col-patch_distance):(patch_mid_col+patch_distance), \
                        (patch_mid_slice-patch_distance):(patch_mid_slice+patch_distance)] = \
                        predicted_mask[(patch_mid_row-patch_distance):(patch_mid_row+patch_distance), \
                        (patch_mid_col-patch_distance):(patch_mid_col+patch_distance), \
                        (patch_mid_slice-patch_distance):(patch_mid_slice+patch_distance)] + \
                            inferred_patch 

            counts[(patch_mid_row-patch_distance):(patch_mid_row+patch_distance), \
                    (patch_mid_col-patch_distance):(patch_mid_col+patch_distance), \
                    (patch_mid_slice-patch_distance):(patch_mid_slice+patch_distance)] \
                    = counts[(patch_mid_row-patch_distance):(patch_mid_row+patch_distance), \
                    (patch_mid_col-patch_distance):(patch_mid_col+patch_distance), \
                    (patch_mid_slice-patch_distance):(patch_mid_slice+patch_distance)] \
                    + np.ones(shape=(patch_size,patch_size,patch_size))

            if patch_mid_row < row_dim - patch_distance:
                # print(patch_mid_row)
                patch_mid_row += patch_size
            else:
                patch_mid_row = patch_distance
            count_patchs += 1
        if patch_mid_col < column_dim - patch_distance:
            patch_mid_col += patch_size
    if patch_mid_slice < slice_dim - patch_distance:
        patch_mid_slice += patch_size

mean_mask = predicted_mask / counts
print(count_patchs)
print(np.mean(mean_mask), np.shape(mean_mask))

patch midpoint location: (32, 32, 32)


2022-04-21 08:39:18.983873: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-21 08:39:19.004298: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699950000 Hz
2022-04-21 08:39:19.158446: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-04-21 08:39:19.968490: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


patch midpoint location: (32, 32, 32)


patch midpoint location: (96, 32, 32)
patch midpoint location: (96, 32, 32)


patch midpoint location: (160, 32, 32)
patch midpoint location: (160, 32, 32)


patch midpoint location: (224, 32, 32)
patch midpoint location: (224, 32, 32)


patch midpoint location: (288, 32, 32)
patch midpoint location: (288, 32, 32)


patch midpoint location: (352, 32, 32)
patch midpoint location: (352, 32, 32)


patch midpoint location: (416, 32, 32)
patch midpoint location: (416, 32, 32)


patch midpoint location: (480, 32, 32)
patch midpoint location: (480, 32, 32)


patch midpoint location: (32, 96, 32)
patch midpoint location: (32, 96, 32)


patch midpoint location: (96, 96, 32)
patch midpoint location: (96, 96, 32)


patch midpoint location: (160, 96, 32)
patch midpoint location: (160, 96, 32)


patch midpoint location: (224, 96, 32)
patch midpoint location: (224, 96, 32)


patch midpoint location: (288, 96, 32)
patch midpoint location: (288, 96, 32)


patch 

/tmp/ipykernel_2166001/1259265627.py:50: RuntimeWarning: invalid value encountered in true_divide
  mean_mask = predicted_mask / counts
